I think it would be cool to look at the embeddings for the subsumption task 

In [40]:
from hierarchy_transformers import HierarchyTransformer
import pandas as pd
import os


In [ ]:
model = HierarchyTransformer.from_pretrained('Hierarchy-Transformers/HiT-MiniLM-L12-WordNetNoun')
model.to(device='mps')


In [57]:
evaluation_path = os.getcwd() + '/scripts/evaluation/hit/hit_eval_local/'
evaluation_results = pd.read_csv(evaluation_path + "results.tsv", sep='\t')
evaluation_results

,Unnamed: 0,centri_weight,threshold,precision,recall,f1,accuracy,accuracy_on_negatives
0,validation,2.0,-13.04,0.877274,0.895494,0.886290,0.979111,0.987472
1,testing,2.0,-13.04,0.876915,0.893568,0.885163,0.978922,0.987458


In [ ]:

# entity names to be encoded.
entity_names = ["laptop", "personal computer", "fruit", "berry"]

# get the entity embeddings
entity_embeddings = model.encode(entity_names)

child_entity_embeddings = model.encode(["laptop", "berry"], convert_to_tensor=True)
parent_entity_embeddings = model.encode(["personal computer", "fruit"], convert_to_tensor=True)

# compute the hyperbolic distances and norms of entity embeddings
dists = model.manifold.dist(child_entity_embeddings, parent_entity_embeddings)
child_norms = model.manifold.dist0(child_entity_embeddings)
parent_norms = model.manifold.dist0(parent_entity_embeddings)

# use the empirical function for subsumption prediction proposed in the paper
# `centri_score_weight` and the overall threshold are determined on the validation set

centri_score_weight = evaluation_results.iloc[0, 1] # use the result on the validation set per the README
threshold = -13.04
subsumption_scores = - (dists + centri_score_weight * (parent_norms - child_norms))

In [67]:
centri_score_weight

np.float64(2.0)

In [33]:
child_entity_embeddings.shape

torch.Size([2, 384])

In [34]:
dists

tensor([12.1125,  6.7239], device='mps:0')

In [35]:
parent_norms, child_norms

(tensor([19.1054, 15.2789], device='mps:0'),
 tensor([21.1878, 16.6117], device='mps:0'))